In [337]:
import pandas as pd
import tabula as tb



In [338]:
year = '2017'
month = '10'

In [339]:
df_row = (tb.read_pdf(f"{year}/PEL/{month}.pdf", pages="all"))[0]
df_row = df_row.rename(columns={"DEMANDA (milhares)": "Linha"})

df_row

,Linha,Linha 1 - Azul,Linha 2 - Verde,Linha 3 - Vermelha,Linha 5 - Lilás,Linha 15 - Prata,Rede
0,Total,26.029,13.219,30.557,6.427,260,76.491
1,Média dos dias úteis,1.059,555.000,1.224,264.000,12,3.114
2,Média dos Sábados,569.000,233.000,684.000,149.000,4,1.637
3,Média dos Domingos,291.000,142.000,405.000,57.000,1,895.000
4,Máxima Diária,1.118,569.000,1.288,280.000,12,3.268


In [340]:
df_row.dtypes

Linha                  object
Linha 1 - Azul        float64
Linha 2 - Verde       float64
Linha 3 - Vermelha    float64
Linha 5 - Lilás       float64
Linha 15 - Prata        int64
Rede                  float64
dtype: object

In [341]:
df_row = df_row.T.reset_index()

In [342]:
df_row.head()

,index,0,1,2,3,4
0,Linha,Total,Média dos dias úteis,Média dos Sábados,Média dos Domingos,Máxima Diária
1,Linha 1 - Azul,26.029,1.059,569.0,291.0,1.118
2,Linha 2 - Verde,13.219,555.0,233.0,142.0,569.0
3,Linha 3 - Vermelha,30.557,1.224,684.0,405.0,1.288
4,Linha 5 - Lilás,6.427,264.0,149.0,57.0,280.0


In [343]:
df_row.columns = ['line', 'total', 'business_day_mean', 'saturday_mean', 'sunday_mean', 'max']

In [344]:
df_row = df_row[1:6]

In [345]:
df_row

,line,total,business_day_mean,saturday_mean,sunday_mean,max
1,Linha 1 - Azul,26.029,1.059,569.0,291.0,1.118
2,Linha 2 - Verde,13.219,555.0,233.0,142.0,569.0
3,Linha 3 - Vermelha,30.557,1.224,684.0,405.0,1.288
4,Linha 5 - Lilás,6.427,264.0,149.0,57.0,280.0
5,Linha 15 - Prata,260,12,4,1,12


In [346]:
df_row['line'] = df_row['line'].apply(lambda x: int(x.split(' ')[1]))

In [353]:
df = df_row.copy()
df

,line,total,business_day_mean,saturday_mean,sunday_mean,max
1,1,26.029,1.059,569.0,291.0,1.118
2,2,13.219,555.0,233.0,142.0,569.0
3,3,30.557,1.224,684.0,405.0,1.288
4,5,6.427,264.0,149.0,57.0,280.0
5,15,260,12,4,1,12


In [354]:
df['total'] = df['total'].apply(lambda x: (int(x) if str(x)[-2:] == '.0' else  int(str(x).replace(',', '').replace('.', '')) if isinstance(x, float) else x) * 1000)
df['business_day_mean'] = df['business_day_mean'].apply(lambda x: (int(x) if str(x)[-2:] == '.0' else  int(str(x).replace(',', '').replace('.', '')) if isinstance(x, float) else x) * 1000)
df['saturday_mean'] = df['saturday_mean'].apply(lambda x: (int(x) if str(x)[-2:] == '.0' else  int(str(x).replace(',', '').replace('.', '')) if isinstance(x, float) else x) * 1000)
df['sunday_mean'] = df['sunday_mean'].apply(lambda x: (int(x) if str(x)[-2:] == '.0' else  int(str(x).replace(',', '').replace('.', '')) if isinstance(x, float) else x) * 1000)
df['max'] = df['max'].apply(lambda x: (int(x) if str(x)[-2:] == '.0' else  int(str(x).replace(',', '').replace('.', '')) if isinstance(x, float) else x) * 1000)
df['date'] = pd.to_datetime(year + '-' + month,format='%Y-%m')

df = df[['date', 'line', 'total', 'business_day_mean', 'saturday_mean', 'sunday_mean', 'max']]
df

,date,line,total,business_day_mean,saturday_mean,sunday_mean,max
1,2017-10-01,1,26029000,1059000,569000,291000,1118000
2,2017-10-01,2,13219000,555000,233000,142000,569000
3,2017-10-01,3,30557000,1224000,684000,405000,1288000
4,2017-10-01,5,6427000,264000,149000,57000,280000
5,2017-10-01,15,260000,12000,4000,1000,12000


In [375]:
def pel2017(year, month): 
    df_row = (tb.read_pdf(f"{year}/PEL/{month}.pdf", pages="all"))[0]

    df_row = df_row.rename(columns={"DEMANDA (milhares)": "Linha"})
    print(df_row.dtypes)
    df = df_row.T.reset_index()
    print(month)
    print(df.dtypes)
    df.columns = ['line', 'total', 'business_day_mean', 'saturday_mean', 'sunday_mean', 'max']

    df = df[1:6]

    #df['line'] = df['line'].apply(lambda x: int(x.split(' ')[1]))
    #df['line'] = df['line'].apply(lambda x: int(x.split(' ')[1]) if ' ' in x else x)
    df['line'] = df['line'].apply(lambda x: int(x.replace('\r', ' ').split(' ')[1]) if ' ' in x else x)



    df['total'] = df['total'].apply(lambda x: (int(x) if str(x)[-2:] == '.0' else  int(str(x).replace(',', '').replace('.', '')) if isinstance(x, float) else x) * 1000)
    df['business_day_mean'] = df['business_day_mean'].apply(lambda x: (int(x) if str(x)[-2:] == '.0' else  int(str(x).replace(',', '').replace('.', '')) if isinstance(x, float) else x) * 1000)
    df['saturday_mean'] = df['saturday_mean'].apply(lambda x: (int(x) if str(x)[-2:] == '.0' else  int(str(x).replace(',', '').replace('.', '')) if isinstance(x, float) else x) * 1000)
    df['sunday_mean'] = df['sunday_mean'].apply(lambda x: (int(x) if str(x)[-2:] == '.0' else  int(str(x).replace(',', '').replace('.', '')) if isinstance(x, float) else x) * 1000)
    df['max'] = df['max'].apply(lambda x: (int(x) if str(x)[-2:] == '.0' else  int(str(x).replace(',', '').replace('.', '')) if isinstance(x, float) else x) * 1000)
    df['date'] = pd.to_datetime(year + '-' + month,format='%Y-%m')

    df = df[['date', 'line', 'total', 'business_day_mean', 'saturday_mean', 'sunday_mean', 'max']]

    return df

In [381]:
year = '2017'
year_df = []
for month in range(10,13):
    df_processed = pel2017(year, str(month))
    year_df.append(df_processed)

Linha                  object
Linha 1 - Azul        float64
Linha 2 - Verde       float64
Linha 3 - Vermelha    float64
Linha 5 - Lilás       float64
Linha 15 - Prata        int64
Rede                  float64
dtype: object
10
index    object
0        object
1        object
2        object
3        object
4        object
dtype: object
Linha                  object
Linha 1 - Azul        float64
Linha 2 - Verde       float64
Linha 3 - Vermelha    float64
Linha 5 - Lilás2      float64
Linha 15 - Prata        int64
Rede                  float64
dtype: object
11
index    object
0        object
1        object
2        object
3        object
4        object
dtype: object
Linha                  object
Linha 1 - Azul        float64
Linha 2 - Verde       float64
Linha 3 - Vermelha    float64
Linha 5 - Lilás2      float64
Linha 15 - Prata        int64
Rede                  float64
dtype: object
12
index    object
0        object
1        object
2        object
3        object
4        object
dty

In [382]:
dataset = pd.concat(year_df)
dataset

,date,line,total,business_day_mean,saturday_mean,sunday_mean,max
1,2017-10-01,1,26029000,1059000,569000,291000,1118000
2,2017-10-01,2,13219000,555000,233000,142000,569000
3,2017-10-01,3,30557000,1224000,684000,405000,1288000
4,2017-10-01,5,6427000,264000,149000,57000,280000
5,2017-10-01,15,260000,12000,4000,1000,12000
1,2017-11-01,1,24834000,1077000,571000,319000,1116000
2,2017-11-01,2,12437000,559000,229000,146000,574000
3,2017-11-01,3,29305000,124000,709000,444000,1274000
4,2017-11-01,5,6348000,274000,155000,77000,285000
5,2017-11-01,15,269000,13000,4000,1000,14000
